<a href="https://colab.research.google.com/github/JimenaBaripatti/FeatureEngineering/blob/main/2_Fire_Incidents_time_related_features_SD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective:
As part of this project, we need to focus on understanding, cleaning, describing, visualizing, and 
transforming data to bring data to life and tell your own version of the story and unveil some hidden insights. 
Primary objective is to come up with creative features using given data and external sources and 
focus less on making a highly accurate model. 

Visualization Tool: You have the option to visualize your charts using Python (library of choice), 
    Tableau, Domo, PowerBI, Google Data Studio or any other visualization tool of choice, 
    so you can focus on feature engineering part.
    
Label: Create a label (target variable) using Civilian_Casualties by converting it into a binary variable 0 / 1 so the modelling problem becomes a classification problem. 
Label = 0 if  Civilian_Casualties = 0
Label = 1 if  Civilian_Casualties > 0

## @Surnjani Djoko Nov 2021
## Notes: this notebook joins incident dataset with fire station dataset, and create time related features plus distance between incident and fire station (km)

## Summary of the output dataset:

In [ ]:
import pandas as pd
import numpy as np
import holidays

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## User defined functions

In [ ]:
# Compute distance between fire location and its fire station
# Distance Feature

def haversine_distance(row):
    '''
    distance is in km
    '''
    lat_p, lon_p = row['Latitude'], row['Longitude'] # fire location
    lat_d, lon_d = row['fs_Longitude'], row['fs_Longitude'] # fire station location
    radius = 6371 # km

    # convert decimal degrees to radians 
    dlat = np.radians(lat_d - lat_p)
    dlon = np.radians(lon_d - lon_p)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat_p)) * np.cos(np.radians(lat_d)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = radius * c

    return distance

# (1) Importing dataset

## (1a) incident dataset

In [ ]:
# main dataset: https://open.toronto.ca/dataset/fire-incidents/
incident=pd.read_csv('group_data/Fire_incidents_Data.csv')
print(incident.shape)
incident.head(3)

(17536, 43)


,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Fire_Under_Control_Time,Ignition_Source,Incident_Number,Incident_Station_Area,Incident_Ward,Initial_CAD_Event_Type,Intersection,Last_TFS_Unit_Clear_Time,Latitude,Level_Of_Origin,Longitude,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties
0,1946929,81 - Engine Area,NaN,NaN,0,0,15000.0,NaN,NaN,2018-02-25T02:12:00,NaN,01 - Fire,NaN,NaN,NaN,2018-02-25T02:15:40,999 - Undetermined,F18020956,441,1.0,Vehicle Fire,Dixon Rd / 427 N Dixon Ramp,2018-02-25T02:38:31,43.686558,NaN,-79.599419,47 - Vehicle,1 - Extinguished by fire department,1,4,99 - Undetermined,"896 - Sidewalk, street, roadway, highway, hwy ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7 - Fully involved (total structure, vehicle, ...",2018-02-25T02:04:29,2018-02-25T02:10:11,0
1,1946930,"75 - Trash, rubbish area (outside)",NaN,NaN,0,0,50.0,NaN,NaN,2018-02-25T02:29:42,NaN,01 - Fire,NaN,NaN,NaN,2018-02-25T02:32:24,999 - Undetermined,F18020969,116,18.0,Fire - Grass/Rubbish,Sheppard Ave E / Clairtrell Rd,2018-02-25T02:35:58,43.766135,NaN,-79.390039,97 - Other,1 - Extinguished by fire department,1,4,03 - Suspected Vandalism,"896 - Sidewalk, street, roadway, highway, hwy ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 - Fire with no evidence from street,2018-02-25T02:24:43,2018-02-25T02:29:31,0
2,1946931,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,"03 - NO LOSS OUTDOOR fire (exc: Sus.arson,vand...",NaN,NaN,NaN,NaN,NaN,F18021182,221,21.0,Fire - Highrise Residential,Danforth Rd / Savarin St,2018-02-25T19:14:03,43.743230,NaN,-79.245061,NaN,NaN,6,22,NaN,891 - Outdoor general auto parking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-25T18:29:59,2018-02-25T18:36:49,0


## (1b) fire station location

In [ ]:
station=pd.read_csv('group_data/fire-station-locations.csv')
print(station.shape)
station.head(3)

(84, 23)


,_id,ID,NAME,ADDRESS,ADDRESS_POINT_ID,ADDRESS_ID,CENTRELINE_ID,MAINT_STAGE,ADDRESS_NUMBER,LINEAR_NAME_FULL,POSTAL_CODE,GENERAL_USE,CLASS_FAMILY_DESC,ADDRESS_ID_LINK,PLACE_NAME,X,Y,LATITUDE,LONGITUDE,WARD_NAME,MUNICIPALITY_NAME,OBJECTID,geometry
0,1,21,FIRE STATION 211,900 TAPSCOTT RD,4236992,363382,4236991,REGULAR,900,Tapscott Rd,NaN,Fire/Ambulance Stations,"Land, Structure, Structure Entrance",NaN,"Fire Station 211, Ambulance Station 27",NaN,NaN,NaN,NaN,Scarborough North (23),Scarborough,1520443,"{u'type': u'Point', u'coordinates': (-79.24287..."
1,2,60,FIRE STATION 342,106 ASCOT AVE,764237,70190,1140634,REGULAR,106,Ascot Ave,NaN,Fire Station,"Land, Structure, Structure Entrance",NaN,Fire Station 342,NaN,NaN,NaN,NaN,Davenport (9),former Toronto,1541526,"{u'type': u'Point', u'coordinates': (-79.44862..."
2,3,61,FIRE STATION 343,65 HENDRICK AVE,819425,127148,1140587,REGULAR,65,Hendrick Ave,NaN,Fire Station,"Land, Structure, Structure Entrance",NaN,Fire Station 343,NaN,NaN,NaN,NaN,Toronto-St. Paul's (12),former Toronto,1543317,"{u'type': u'Point', u'coordinates': (-79.43075..."


In [ ]:
# extract longitude and latitude from geometry column
station['fs_Longitude'] = station['geometry'].str.extract('([-]?[0-9]*[.]{0,1}[0-9]{10})').astype(float)
station['fs_Latitude'] = station['geometry'].str[53:-2].astype(float)

In [ ]:
# extract station area so it can be used to join with incident dataset
station['Station_Area']= station.NAME.str.extract('(\d+)')
station['Station_Area'] = station['Station_Area'].apply(pd.to_numeric)

In [ ]:
incident.Incident_Station_Area.unique()

array([441, 116, 221, 133, 132, 215, 235, 231, 332, 426, 225, 325, 226,
       341, 421, 244, 141, 115, 415, 431, 331, 413, 314, 333, 311, 145,
       143, 342, 443, 312, 223, 134, 214, 434, 423, 233, 114, 112, 224,
       326, 212, 343, 135, 125, 315, 234, 324, 113, 142, 146, 313, 442,
       222, 241, 345, 232, 121, 432, 425, 334, 411, 445, 243, 323, 435,
       213, 422, 412, 123, 344, 111, 242, 321, 433, 245, 211, 131, 322,
       444, 122, 335, 227, 346, 424])

In [ ]:
incident = pd.merge(incident,station, how='left',left_on='Incident_Station_Area', right_on = 'Station_Area')

In [ ]:
print(incident.shape)
incident.sample(3)

(17536, 69)


,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Fire_Under_Control_Time,Ignition_Source,Incident_Number,Incident_Station_Area,Incident_Ward,Initial_CAD_Event_Type,Intersection,Last_TFS_Unit_Clear_Time,Latitude,Level_Of_Origin,Longitude,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties,_id_y,ID,NAME,ADDRESS,ADDRESS_POINT_ID,ADDRESS_ID,CENTRELINE_ID,MAINT_STAGE,ADDRESS_NUMBER,LINEAR_NAME_FULL,POSTAL_CODE,GENERAL_USE,CLASS_FAMILY_DESC,ADDRESS_ID_LINK,PLACE_NAME,X,Y,LATITUDE,LONGITUDE,WARD_NAME,MUNICIPALITY_NAME,OBJECTID,geometry,fs_Longitude,fs_Latitude,Station_Area
15689,1962618,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0,0,1000.0,0.0,NaN,2011-12-29T08:03:22,2 - Confined to part of room/area of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",8 - Not applicable (no system),8 - Not applicable (bldg not classified by OBC...,2011-12-29T08:03:22,33 - Space Heater - Fixed,F11164521,131,16.0,FIR,Greer Rd / Glengarry Ave,2011-12-29T08:44:35,43.724460,001,-79.409070,"36 - Rug, Carpet",3 - Extinguished by occupant,5,17,52 - Electrical Failure,302 - Semi-Detached Dwelling,2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,1 - Battery operated,1 - All persons (at risk of injury) self evacu...,3 - Spread to entire room of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2011-12-29T07:58:35,2011-12-29T08:03:10,0,30,11,FIRE STATION 131,3135 YONGE ST,7785457,420576,7785453,REGULAR,3135,Yonge St,NaN,Fire Station,"Land, Structure, Structure Entrance",NaN,Fire Station 131,NaN,NaN,NaN,NaN,Don Valley West (15),former Toronto,2119715,"{u'type': u'Point', u'coordinates': (-79.40215...",-79.402157,43.726217,131
3161,1950090,64 - Porch or Balcony,01 - Normal (no change),1 - No business interruption,1,0,5000.0,0.0,NaN,2018-12-07T20:38:48,2 - Confined to part of room/area of origin,01 - Fire,3 - No one (at risk) evacuated as a result of ...,1 - Fire alarm system operated,1 - Fire alarm system present,2018-12-07T20:36:01,"71 - Smoker's Articles (eg. cigarettes, cigars...",F18130661,112,6.0,Fire - Highrise Residential,Fisherville Rd / Carpenter Rd,2018-12-07T21:39:33,43.790188,010,-79.447109,33 - Linen Other than Bedding,1 - Extinguished by fire department,17,53,45 - Improperly Discarded,323 - Multi-Unit Dwelling - Over 12 Units,2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,1 - Battery operated,1 - All persons (at risk of injury) self evacu...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,4 - Flames showing from small area (one storey...,2018-12-07T20:25:06,2018-12-07T20:29:54,0,73,2,FIRE STATION 112,5700 BATHURST ST,9313399,413809,14661127,REGULAR,5700,Bathurst St,NaN,Fire/Ambulance Stations,"Land, Structure, Structure Entrance",NaN,"Fire Station 112, Ambulance Station 55",NaN,NaN,NaN,NaN,York Centre (6),North York,3311629,"{u'type': u'Point', u'coordinates': (-79.44428...",-79.444286,43.776286,112
4159,1951088,"21 - Living Area (e.g. living, TV, recreation,...",01 - Normal (no change),1 - No business interruption,0,0,5000.0,0.0,NaN,2018-07-04T00:01:00,1 - Confined to object of or

## process target

In [ ]:
incident['LABEL'] = 0
incident.loc[incident.Civilian_Casualties > 0, 'LABEL']=1

In [ ]:
pd.crosstab(incident.LABEL, incident.Civilian_Casualties)

Civilian_Casualties,0,1,2,3,4,5,6,7,8,15
LABEL,,,,,,,,,,
0,16543,0,0,0,0,0,0,0,0,0
1,0,807,116,40,20,2,4,1,2,1


In [ ]:
incident.LABEL.value_counts(normalize=True)

0    0.943374
1    0.056626
Name: LABEL, dtype: float64

## (2) Reformat date related columns

In [ ]:
#Convert the string to datetime format

date_var = ["TFS_Alarm_Time", "TFS_Arrival_Time", "Fire_Under_Control_Time", "Last_TFS_Unit_Clear_Time"]
incident[date_var] = incident[date_var].apply(pd.to_datetime)

## (3.1) Create distance feature

In [ ]:
# what is the distance between incident and fire station (in km)
incident['DISTANCE_INCIDENT_FIRESTATION'] = incident.apply(haversine_distance, axis = 1)

## (3.2) Create dates related features
### minutes to arrive, minutes to leave, incident period of the day, is holiday, and a day or a day after is a holidy

In [ ]:
incident['INCIDENT_DATE']= incident['TFS_Alarm_Time'].apply(lambda x: x.date())
incident['INCIDENT_DATE'] = incident['INCIDENT_DATE'].apply(pd.to_datetime)
incident['DOW']= incident['INCIDENT_DATE'].apply(lambda x: x.weekday())
incident['IS_WEEKEND'] = incident['INCIDENT_DATE'].apply(lambda x: 1 if x.weekday() in (5, 6) else 0)

In [ ]:
incident['YEAR'] = pd.DatetimeIndex(incident['INCIDENT_DATE']).year
incident['MONTH'] = pd.DatetimeIndex(incident['INCIDENT_DATE']).month

In [ ]:
incident.sample(3)

,_id_x,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Fire_Under_Control_Time,Ignition_Source,Incident_Number,Incident_Station_Area,Incident_Ward,Initial_CAD_Event_Type,Intersection,Last_TFS_Unit_Clear_Time,Latitude,Level_Of_Origin,Longitude,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties,_id_y,ID,NAME,ADDRESS,ADDRESS_POINT_ID,ADDRESS_ID,CENTRELINE_ID,MAINT_STAGE,ADDRESS_NUMBER,LINEAR_NAME_FULL,POSTAL_CODE,GENERAL_USE,CLASS_FAMILY_DESC,ADDRESS_ID_LINK,PLACE_NAME,X,Y,LATITUDE,LONGITUDE,WARD_NAME,MUNICIPALITY_NAME,OBJECTID,geometry,fs_Longitude,fs_Latitude,Station_Area,LABEL,DISTANCE_INCIDENT_FIRESTATION,INCIDENT_DATE,DOW,IS_WEEKEND,YEAR,MONTH
4417,1951346,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,"03 - NO LOSS OUTDOOR fire (exc: Sus.arson,vand...",NaN,NaN,NaN,NaT,NaN,F18047900,141,7.0,Fire - Highrise Residential,Fountainhead Rd / Sentinel Rd,2018-05-08 17:48:59,43.762241,NaN,-79.500521,NaN,NaN,6,21,NaN,323 - Multi-Unit Dwelling - Over 12 Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-08 17:20:03,2018-05-08 17:22:46,0,80,16,FIRE STATION 141,4100 KEELE ST,30013624,1494773,30020840,REGULAR,4100,Keele St,NaN,Fire/Ambulance Stations,"Land, Structure, Structure Entrance",NaN,Fire Station 141,NaN,NaN,NaN,NaN,Humber River-Black Creek (7),North York,4149403,"{u'type': u'Point', u'coordinates': (-79.49272...",-79.492724,43.766822,141,0,13705.326755,2018-05-08,1,0,2018,5
9946,1956875,81 - Engine Area,NaN,NaN,0,0,1000.0,NaN,NaN,2014-12-02T02:00:00,NaN,01 - Fire,NaN,NaN,NaN,2014-12-02 02:03:00,999 - Undetermined,F14106059,233,34.0,VEF,Roanoke Rd / Underhill Dr,2014-12-02 02:39:33,43.741930,NaN,-79.325200,99 - Undetermined (formerly 98),1 - Extinguished by fire department,1,4,20 - Design/Construction/Installation/Maintena...,901 - Automobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 - Flames showing from small area (one storey...,2014-12-02 01:52:21,2014-12-02 01:58:44,0,54,35,FIRE STATION 233,59 CURLEW DR,8292449,314198,8292440,REGULAR,59,Curlew Dr,NaN,Fire Station,"Land, Structure, Structure Entrance",NaN,Fire Station 233,NaN,NaN,NaN,NaN,Don Valley East (16),North York,2949111,"{u'type': u'Point', u'coordinates': (-79.31452...",-79.314520,43.744103,233,0,13683.252964,2014-12-02,1,0,2014,12
3387,1950316,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,"03 - NO LOSS OUTDOOR fire (exc: Sus.arson,vand...",NaN,NaN,NaN,NaT,NaN,F18112802,114,18.0,Fire - Grass/Rubbish,Yonge St / Ellerslie Ave,2018-10-20 20:48:39,43.771992,NaN,-79.413575,NaN,NaN,1,4,NaN,848 - Trash/Rubbish/Garbage Container or Dumpster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-20 20:37:08,2018-10-20 20:40:47,0,75,4,FIRE STATION 114,12 CANTERBURY PL,13498086,414669,438636,REGULAR,12,Canterbury Pl,NaN,Fire/Ambulance Stations,"Land, Structure, Structure Entrance",NaN,"Fire Station 114, Ambulance Station 58",NaN,NaN,NaN,NaN,Willowdale (18),North York,3311726,"{u'type': u'Point', u'coordinates': (-79.41535...",-79.415355,43.772636,114,0,13697.808038,2018-10-20,5,1,2018,10


In [ ]:
# how long does it take for fire services to arrive at the incident location
# get the number of minutes it took for Fire Services to arrive to the emergency
incident["MINUTES_TO_ARRIVE"] = np.around((incident["TFS_Arrival_Time"] - 
                            incident["TFS_Alarm_Time"]) / np.timedelta64(1, "m"), decimals=3)

In [ ]:
incident['TFS_ARR_DAY'] = incident['TFS_Arrival_Time'].dt.day
incident['TFS_ALM_DAY'] = incident['TFS_Alarm_Time'].dt.day

incident['TFS_ARR_HOUR'] = incident['TFS_Arrival_Time'].dt.hour
incident['TFS_ALM_HOUR'] = incident['TFS_Alarm_Time'].dt.hour

In [ ]:
# Once the fires crew arrives, how long does it take to clear the fire
incident["MINUTES_TO_LEAVE"] = np.around((incident["Last_TFS_Unit_Clear_Time"] -
              incident["TFS_Arrival_Time"]) / np.timedelta64(1, "m"), decimals=3)

In [ ]:
incident["TFS_Alarm_Time"].min()

Timestamp('2011-01-01 05:06:48')

In [ ]:
incident["TFS_Alarm_Time"].max()

Timestamp('2019-07-01 03:47:46')

In [ ]:
# create categorical feature to indicate the incident period 
day_mapping = {1: 'Late Night',
               2: 'Early Morning',
               3: 'Morning',
               4: 'Noon',
               5: 'Evening',
               6: 'Night'}
incident['INCIDENT_PERIOD_NUM'] = (incident['TFS_ALM_HOUR']%24+4)//4
incident['INCIDENT_PERIOD_CAT'] = incident['INCIDENT_PERIOD_NUM'].replace(day_mapping)

In [ ]:
pd.crosstab(incident.INCIDENT_PERIOD_NUM,incident.INCIDENT_PERIOD_CAT)

INCIDENT_PERIOD_CAT,Early Morning,Evening,Late Night,Morning,Night,Noon
INCIDENT_PERIOD_NUM,,,,,,
1,0,0,3531,0,0,0
2,2186,0,0,0,0,0
3,0,0,0,1561,0,0
4,0,0,0,0,0,2405
5,0,3611,0,0,0,0
6,0,0,0,0,4242,0


In [ ]:
incident.INCIDENT_PERIOD_NUM.value_counts(normalize=True)

6    0.241902
5    0.205919
1    0.201357
4    0.137146
2    0.124658
3    0.089017
Name: INCIDENT_PERIOD_NUM, dtype: float64

In [ ]:
# prep for the holiday calculation
years = incident['INCIDENT_DATE'].dt.year.unique()
years = sorted(years)
years_ = [int(item) for item in years]

In [ ]:
years = incident['INCIDENT_DATE'].dt.year.unique()
CA_holidays = []
for year in years_:
    for date in holidays.CA(prov='ON',years=year).items():
        CA_holidays.append(str(date[0]))
print(CA_holidays)

['2011-01-01', '2010-12-31', '2011-02-21', '2011-04-22', '2011-05-23', '2011-07-01', '2011-08-01', '2011-09-05', '2011-10-10', '2011-12-25', '2011-12-26', '2011-12-27', '2012-01-01', '2012-01-02', '2012-02-20', '2012-04-06', '2012-05-21', '2012-07-01', '2012-07-02', '2012-08-06', '2012-09-03', '2012-10-08', '2012-12-25', '2012-12-26', '2013-01-01', '2013-02-18', '2013-03-29', '2013-05-20', '2013-07-01', '2013-08-05', '2013-09-02', '2013-10-14', '2013-12-25', '2013-12-26', '2014-01-01', '2014-02-17', '2014-04-18', '2014-05-19', '2014-07-01', '2014-08-04', '2014-09-01', '2014-10-13', '2014-12-25', '2014-12-26', '2015-01-01', '2015-02-16', '2015-04-03', '2015-05-18', '2015-07-01', '2015-08-03', '2015-09-07', '2015-10-12', '2015-12-25', '2015-12-28', '2016-01-01', '2016-02-15', '2016-03-25', '2016-05-23', '2016-07-01', '2016-08-01', '2016-09-05', '2016-10-10', '2016-12-25', '2016-12-26', '2016-12-27', '2017-01-01', '2017-01-02', '2017-02-20', '2017-04-14', '2017-05-22', '2017-07-01', '2017

In [ ]:
incident['IS_HOLIDAY'] = [1 if str(val).split()[0] in CA_holidays else 0 for val in incident['INCIDENT_DATE']]

In [ ]:
# 1 day before holiday
incident['IS_HOLIDAY_LAG1'] = [1 if str(val).split()[0] in CA_holidays \
                               else 0 for val in (incident['INCIDENT_DATE']+pd.Timedelta(days=-1))]
# 1 day after holiday
incident['IS_HOLIDAY_LEAD1'] = [1 if str(val).split()[0] in CA_holidays \
                               else 0 for val in (incident['INCIDENT_DATE']+pd.Timedelta(days=1))]

In [ ]:
def is_holiday_season(row):
    row_sum = row['IS_HOLIDAY'] + row['IS_HOLIDAY_LEAD1'] + row['IS_HOLIDAY_LAG1']
    return (1 if row_sum >= 1 else 0)

In [ ]:
incident['IS_HOLIDAY_SEASON'] = incident.apply(is_holiday_season, axis=1)

In [ ]:
incident[['IS_HOLIDAY','IS_HOLIDAY_LAG1','IS_HOLIDAY_LEAD1','IS_HOLIDAY_SEASON']].nunique()

IS_HOLIDAY           2
IS_HOLIDAY_LAG1      2
IS_HOLIDAY_LEAD1     2
IS_HOLIDAY_SEASON    2
dtype: int64

In [ ]:
incident.to_csv('group_data/incident_SD.csv', index=False)

## Quick check on the data

In [ ]:
incident.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17536 entries, 0 to 17535
Data columns (total 88 columns):
 #   Column                                                         Non-Null Count  Dtype         
---  ------                                                         --------------  -----         
 0   _id_x                                                          17536 non-null  int64         
 1   Area_of_Origin                                                 15623 non-null  object        
 2   Building_Status                                                11216 non-null  object        
 3   Business_Impact                                                11214 non-null  object        
 4   Civilian_Casualties                                            17536 non-null  int64         
 5   Count_of_Persons_Rescued                                       17536 non-null  int64         
 6   Estimated_Dollar_Loss                                          15627 non-null  float64       


In [ ]:
# Deterime the pecentage of nan per column
incident.isnull().sum()/incident.shape[0]*100

_id                                                               0.000000
Area_of_Origin                                                   10.908987
Building_Status                                                  36.040146
Business_Impact                                                   0.000000
Civilian_Casualties                                               0.000000
Count_of_Persons_Rescued                                          0.000000
Estimated_Dollar_Loss                                            10.886177
Estimated_Number_Of_Persons_Displaced                            36.045849
Exposures                                                        98.101049
Ext_agent_app_or_defer_time                                      10.914690
Extent_Of_Fire                                                    0.000000
Final_Incident_Type                                               0.000000
Fire_Alarm_System_Impact_on_Evacuation                           36.051551
Fire_Alarm_System_Operati

In [ ]:
# ##################